In [1]:
import requests, time, sys
from bs4 import BeautifulSoup

In [2]:
# Function to attempt to retrieve
def get_attempt(base_url, get_params=None, attempts=10):
    for attempt in range(attempts):
        time.sleep(0.5)
        try:
            response = requests.get(base_url, params=get_params, allow_redirects=True)
            break
        except:
            if attempt == attempts - 1:
                response = "FAILED"
            continue
    if response == "FAILED":
        print(f"Could not search {base_url}")
    return response

In [3]:
# Recursive function to check page for targets and crawl unseen links
def check_page(href, done_hrefs):
    done_hrefs.append(href)
    req = get_attempt(done_hrefs[0] + href)
    if req == "FAILED":
        return
    html = req.content
    soup = BeautifulSoup(html, 'html.parser')
    anchors = soup.find_all("a")
    for item in anchors:
        if 'href' in item.attrs.keys():
            if (item['href'] not in done_hrefs) and (item['href'][:4] != "http") and (item['href'].find('?') == -1):
                check_page(item['href'], done_hrefs, targets, hit_hrefs)
    return

In [4]:
# Initial inputs - root and target(s) supplied by user
root_url = input("Please enter a root url: ")
done_hrefs = [root_url, '/']

Please enter a root url:  http://argleblargle.com


In [5]:
# Attempt to access user-supplied root url and generate initial link list to work from
r = get_attempt(root_url)
if r == "FAILED":
    print("Failed to access root url")
    sys.exit()
root_html = r.content
root_soup = BeautifulSoup(root_html, 'html.parser')
home_links = root_soup.find_all('a')

home_hrefs = []
for link in home_links:
    if ('href' in link.attrs.keys()) and (link['href'] not in home_hrefs) and (link['href'].find('?') == -1) and (link['href'][:4] != "http"):
        home_hrefs.append(link['href'])

Could not search http://argleblargle.com
Failed to access root url


AttributeError: 'str' object has no attribute 'content'

In [ ]:
# Iterate over initial link list and recurse to crawl through site - pages with instances of targets are printed
for home_href in home_hrefs:
    print(f"Now processing {home_hrefs.index(home_href)+1} of {len(home_hrefs)}, {home_href}")
    check_page(home_href, done_hrefs, targets, hit_hrefs)

In [ ]:
# Total number of pages with instances of targets
print(len(hit_hrefs))

In [ ]:
# Final output
for hit in hit_hrefs:
    print(f"{root_url + hit[0]}, {hit[1]}")